In [1]:
import numpy as np
f = open("main.cpp","w+")
wgtIdx = 0
thIdx = 0
act_name = "bin_img"

In [4]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


# load image using PIL
img = Image.open("7.png")
# convert to black and white
img = img.convert("L")
# convert to numpy array
img = np.asarray(img)
# display
% matplotlib inline
#plt.imshow(img, cmap='gray')

from QNN.layers import *
import pickle
# load the qnn
qnn = pickle.load(open("mnist-w1a1.pickle", "rb"))
new = []
prev_name = ''
prev = qnn[0]
#the new list will have just pairs fullyconnected-thresholding
for l in qnn:
    if(prev_name == 'QNNFullyConnectedLayer'and l.layerType() == 'QNNBipolarThresholdingLayer'):
        new.append(prev)
        new.append(l)
        print(l.layerType())
    prev_name = l.layerType()
    prev = l
new

print(new[0].W.shape)
print(new[1].thresholds.shape)
predict(qnn[0:1], img).shape

QNNBipolarThresholdingLayer
QNNBipolarThresholdingLayer
QNNBipolarThresholdingLayer
(256, 784)
(1, 256)
(1, 784)


(784,)

In [19]:
f.write("\
#include \"BitSerialMatMulAccelDriver.hpp\"\n\
#include \"myfancyNewHeaderFile.hpp\"\n\
\n\
int main(int argc, char * argv[]){\n\
\
}\n\
")

In [20]:
f.close() 

In [15]:
print("\
#include \"BitSerialMatMulAccelDriver.hpp\"\n\
#include \"myfancyNewHeaderFile.hpp\"\n\
#include \"cnpy.h\"\n\
#include <vector>\n\
#include <iostream>\n\
#include <assert.h>\n\
#include \"utils.hpp\"\n\
using namespace gemmbitserial;\n\
\n\
int main(int argc, char * argv[]){\n\
  try {\n\
    WrapperRegDriver * platform = initPlatform();\n\
    BitSerialMatMulAccelDriver * acc_driver = new BitSerialMatMulAccelDriver(platform);\n\
    \n\\n\
    //load activations from file\n\
    cnpy::NpyArray activations_npy = cnpy::npy_load(\""+ str(act_name) +".npy\");\n\
    uint8_t * activations_ptr = activations_npy.data<uint8_t>();\n\
    size_t act_size = activations_npy.shape.size();\n\
    assert(act_size == 2);\n\
    size_t rows_rhs = activations_npy.shape[0];\n\
    size_t cols_rhs = activations_npy.shape[1];\n\
    size_t act_bytes = rows_rhs * cols_rhs;\n\
    \n\n\
    //for each test layer (pair fully connected-thresholding) \n\n\
    //weights loading  \n\
        cnpy::NpyArray weights_npy = cnpy::npy_load(\"weights_l"+ str(wgtIdx)+".npy\");\n\
        uint8_t* weights_ptr = weights_npy.data<uint8_t>();\n\
        int32_t wshape_size = weights_npy.shape.size();\n\
        assert(wshape_size == 2);\n\
        std::vector<size_t> wshape;\n\
        int32_t wgt_bytes = 1;\n\
        for(int i = 0; i < wshape_size; i++){\n\
            wshape.push_back(weights_npy.shape[i]);\n\
            wgt_bytes *= weights_npy.shape[i];\n\
        }\n\n\
        //prepare gemm bit serial\n\
        size_t rows_lhs = wshape[0];\n\
        size_t cols_lhs = wshape[1];\n\
        //COMMENT: the vector creation is not neeeded, could manipulate directly pointer\n\
        uint8_t * lhs = new uint8_t[wgt_bytes];\n\
        uint8_t * rhs = new uint8_t[act_bytes];\n\
        memcpy(lhs,weights_ptr, rows_lhs * cols_lhs * sizeof(uint8_t));\n\
        memcpy(rhs, activations_ptr, rows_rhs * cols_rhs * sizeof(uint8_t));\n\
        uint8_t * rhs_t = new uint8_t[act_bytes];\n\
        //Be compliant with gemmbitserial hypothesis that rhs always transposed\n\
        transpose(activations_ptr, rows_rhs, cols_rhs, rhs_t, cols_rhs, rows_rhs);\n\
        //Allocate gemmbitserial\n\
        //TODO still miss bits and sign, now no check on matrices sizes\n\
        //p2S?\n\
        GEMMContext ctx = acc->allocGEMMContext(\n\
            rows_lhs, ncols, rows_rhs, nbits_lhs, nbits_rhs, sgn_lhs, sgn_rhs\n\
        );\n\
        ctx.lhs.importRegular(lhs);\n\
        ctx.rhs.importRegular(rhs);\n\n\
        //Allocate buffer for lhs and rhs\n\
        //TODO: fixed size a priori?(now nope)\n\
        size_t res_bytes = rows_lhs * rows_rhs * sizeof(uint8_t);\n\
        void * lhs_base_addr = platform->allocAccelBuffer(wgt_bytes);\n\
        void * rhs_base_addr = platform->allocAccelBuffer(act_bytes);\n\
        void * res_base_addr = platform->allocAccelBuffer(res_bytes);\n\
        platform->copyBufferHostAccel(ctx.lhs, lhs_base_addr, wgtbytes);\n\
        //TODO: Invalidate cache?\n\
        platform->copyBufferHostAccel(ctx.rgs, rhs_base_addr, act_bytes);\n\n\
    //thresholds loading \n\
        cnpy::NpyArray ths_npy = cnpy::npy_load(\"ths_l"+ str(thIdx)+".npy\");\n\
        int32_t * ths_ptr = ths_npy.data<int32_t>();\n\
        size_t thshape_size = ths_npy.shape.size();\n\
        // which layout? ths[thnmbr][rows_lhs]? ths[rows_lhs][thnumber]? \n\
        // the thresholding stage is in the second layout\n\
        assert(thshape_size == 2);\n\
        //loading ths\n\
        //iteration depends on the layout\n\
        // loop on(follow an example) acc_driver->thsSingleTransfer(&ths[i * ncols_ths + j], addr_offset , i, j);\n\
        //addr_offset bram address, i-j determine which is the target bram among the possible i-j\n\
        //This loading should also take care of possible folding in the different dimensions\n\
        int32_t addr_offset = 0;\n\
        int32_t dpa_lhs = acc_driver->getdpaDimLHS();\n\
        for (int i = 0; i < rows_lhs; i++){\n\
            for(int j = 0; j < thnumber; j++ ){\n\
                acc_driver->thsSingleTransfer(&ths_bs[i * thnumber + j], addr_offset , i, j);\n\
            }\n\
            if((i+1)%dpa_lhs == 0){\n\
            addr_offset ++;\n\
            }\n\
          }\n\n\
        // Act, Weights, Thresholds all load offload to accel\n\
        offloadMVT_Preloaded(weights, wgtprec, activations, actprec, thresholds, thnumber, res_base_addr);\n\
        // golden results computation\n\
        int32_t * qres = new int32_t[res_bytes]\n\
        gemmBitSerial(ctx);\n\
        quantizeMatrix(ctx.res, ths, rows_lhs, rows_rhs, rows_lhs, thnmbr, qres, 0);\n\
        uint8_t * hw_res = new uint8_t[res_bytes]\n\
        //we should take care of which output is produced uint8_t/int32_t...\n\
        platform->copyBufferAccelToHost(res_base_addr, hw_res, res_bytes)\n\
        memcmp(hw_res, qres, res_bytes);\n\
    }\n\
    platform->deallocAccelBuffer(lhs_base_addr);\n\
    platform->deallocAccelBuffer(rhs_base_addr);\n\
    platform->deallocAccelBuffer(res_base_addr);\n\
    delete acc_driver;\n\
    deinitPlatform(platform);\n\
  } catch (const char * e) {\n\
    std::cout << \"Exception: \" << e << std::endl;\n\
  }\n\
  return 0;\n\
}\n\
")

#include "BitSerialMatMulAccelDriver.hpp"
#include "myfancyNewHeaderFile.hpp"
#include "cnpy.h"
#include <vector>
#include <iostream>
#include <assert.h>
#include "utils.hpp"
using namespace gemmbitserial;

int main(int argc, char * argv[]){
  try {
    WrapperRegDriver * platform = initPlatform();
    BitSerialMatMulAccelDriver * acc_driver = new BitSerialMatMulAccelDriver(platform);
    
\n    //load activations from file
    cnpy::NpyArray activations_npy = cnpy::npy_load("bin_img.npy");
    uint8_t * activations_ptr = activations_npy.data<uint8_t>();
    size_t act_size = activations_npy.shape.size();
    assert(act_size == 2);
    size_t rows_rhs = activations_npy.shape[0];
    size_t cols_rhs = activations_npy.shape[1];
    size_t act_bytes = rows_rhs * cols_rhs;
    

    //for each test layer (pair fully connected-thresholding) 

    //weights loading  
        cnpy::NpyArray weights_npy = cnpy::npy_load("weights_l0.npy");
        uint8_t* weights_ptr = weights_npy.data<uint8_t